## Some Extra Needed Codes
Just for organizing files...

### Imports and Variables

In [1]:
import os
import re

# directory = 'Extra_Codes_Directory'
directory = 'Voices'
os.makedirs(directory, exist_ok=True)

### File Renaming

In [ ]:
def rename_files(start_number, directory):
    current = start_number

    while True:
        old_filename = os.path.join(directory, f"{current}.m4a")
        new_filename = os.path.join(directory, f"{current - 1}.m4a")

        if os.path.exists(old_filename):
            print(f"Renaming {old_filename} → {new_filename}")
            os.rename(old_filename, new_filename)
            current += 1
        else:
            print(f"No file found: {old_filename}. Stopping.")
            break

# Example usage:
rename_files(8, "Voices")

### Missed Numbers in Voice Files

In [ ]:
# pattern = re.compile(r'^(\d+)\.m4a$') # 1.m4a, 2.m4a, etc.
pattern = re.compile(r'^Voice (\d+)\.m4a$') # Voice 1.m4a, Voice 2.m4a, etc.

numbers = []

for filename in os.listdir(directory):
    match = pattern.match(filename)
    if match:
        numbers.append(int(match.group(1)))

if not numbers:
    print("No .m4a files with numeric names found.")
else:
    numbers = sorted(numbers)
    missing = []

    for num in range(numbers[0], numbers[-1] + 1):
        if num not in numbers:
            missing.append(num)

    if missing:
        print("Missing numbers:", missing)
    else:
        print("No missing numbers. Sequence is complete!")


No missing numbers. Sequence is complete!


### Shift Numbers in File Names

In [ ]:
pattern = re.compile(r'(Voice )(\d{3})(\.m4a)')

for filename in os.listdir(directory):
    match = pattern.match(filename)
    if match:
        prefix, number_str, suffix = match.groups()
        new_number = int(number_str) + 410
        new_filename = f"{prefix}{new_number:03d}{suffix}"
        old_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")

print("✅ Done renaming all matching files!")
